# Bags within bags

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:
```
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
```
These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

A bright white bag, which can hold your shiny gold bag directly.
A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

## Second part

It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

- faded blue bags contain 0 other bags.
- dotted black bags contain 0 other bags.
- vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
- dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags. 

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): `1 + 1*7 + 2 + 2*11 = 32` bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:
```
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
```
In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?

In [1]:
import os
import time
import numpy as np
import re

In [2]:
DAY = 'Day_07'
FILE_END = '_01.txt'
RUN_WITH = 'input' ## admitted values: sample input 
SAMPLE_DATA = '''light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.'''

In [3]:
start = time.time()

In [4]:

input_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))), "Inputs")
input_path_day = os.path.join(input_path, DAY)
file_path = os.path.join(input_path_day, DAY+ FILE_END)

### Read input

In [5]:
if RUN_WITH == 'input':
    with open(file_path,'r') as f:
        values = f.read()
elif RUN_WITH == 'sample':
    values =  SAMPLE_DATA
    
else:
    print("No valid input selected")
values = values.split('\n')
values = [x.replace('no other','0 other').replace('bags','').replace('bag','').replace('.','').strip() for x in values]
try:
    values.remove("")
except:
    pass
values[0:5]

['striped tan  contain 2 light silver , 1 drab black , 2 clear tan , 2 mirrored tan',
 'dark black  contain 1 vibrant indigo , 5 muted gold , 4 bright tomato , 3 dull tan',
 'dim silver  contain 1 vibrant black , 3 muted cyan , 4 plaid turquoise , 4 faded orange',
 'faded maroon  contain 3 pale aqua',
 'dim crimson  contain 1 faded orange , 2 plaid silver']

## Get Rules

In [6]:
def parse_line(line):
    ls = [x.strip() for x in line.split('contain')]
    key = ls[0]
    value = [{'bag':' '.join(s.split()[1:]) , 'q': int(s.split()[0])} for s in ls[1].split(',')]
    return key, value

In [7]:
keys = {}
for line in values:
    try:
        k,v = parse_line(line)
        keys[k] = v
    except:
        print(f'Failed at line: {line}')

In [8]:
# Sample 
list_keys = [key for key in keys.keys()]
for k in list_keys[0:5]:
    print(k, keys[k])

striped tan [{'bag': 'light silver', 'q': 2}, {'bag': 'drab black', 'q': 1}, {'bag': 'clear tan', 'q': 2}, {'bag': 'mirrored tan', 'q': 2}]
dark black [{'bag': 'vibrant indigo', 'q': 1}, {'bag': 'muted gold', 'q': 5}, {'bag': 'bright tomato', 'q': 4}, {'bag': 'dull tan', 'q': 3}]
dim silver [{'bag': 'vibrant black', 'q': 1}, {'bag': 'muted cyan', 'q': 3}, {'bag': 'plaid turquoise', 'q': 4}, {'bag': 'faded orange', 'q': 4}]
faded maroon [{'bag': 'pale aqua', 'q': 3}]
dim crimson [{'bag': 'faded orange', 'q': 1}, {'bag': 'plaid silver', 'q': 2}]


## Part I

In [9]:
def list_with_target(target):
    return [key for key in keys.keys() for items in keys[key] if items['bag'] == target]

In [10]:
target = 'shiny gold'
processed = set()
to_process = [target]
matches = []

In [11]:
while to_process:
    target = to_process.pop()
    matches.append(target)
    if target not in processed:
        to_process.extend(list_with_target(target))
        processed.add(target)
        

In [12]:
len(set(matches)) -1

302

## Part II

In [13]:
def return_children(list_item):
    target, q = list_item
    return q * [(x['bag'],x['q']) for x in keys.get(target, [])], q

In [14]:
# Sample
return_children(('shiny gold', 1))

([('dark brown', 2),
  ('mirrored coral', 2),
  ('faded olive', 1),
  ('posh bronze', 1)],
 1)

In [15]:
count = 0

In [16]:
target = ('shiny gold',1)
processed = set()
children = [target]
matches = []

In [17]:
while children:
    child = children.pop()
    if child[0] != 'other':
        new_children, ct = return_children(child)
        count += ct
        children.extend(new_children)

In [18]:
count-1

4165

In [19]:
end = time.time()
end - start

0.20792102813720703